# Getting Started With 3D Data in Scale Nucleus

---



In this tutorial, we'll walk through a demo of how to upload 3D Scenes to Nucleus using the open source PandaSet dataset.

### Step 1: Install dependencies

In [1]:
%%bash
pip install scale-nucleus
pip install boto3
pip install botocore

You should consider upgrading via the '/Users/drewkaul/Library/Caches/pypoetry/virtualenvs/scale-nucleus-nqXix_D8-py3.8/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/drewkaul/Library/Caches/pypoetry/virtualenvs/scale-nucleus-nqXix_D8-py3.8/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/drewkaul/Library/Caches/pypoetry/virtualenvs/scale-nucleus-nqXix_D8-py3.8/bin/python -m pip install --upgrade pip' command.


### Step 2: Read PandaSet files from S3

More information about PandaSet can be found at: https://scale.com/open-datasets/pandaset

For the purposes of this demo, we've pre-processed the PandaSet dataset into a more convenient form and made it accessible via a public S3 bucket. PandaSet is licensed under the following terms of use: https://scale.com/legal/pandaset-terms-of-use

In [2]:
import os
import re
import json
import boto3
from typing import List
from botocore import UNSIGNED
from botocore.client import Config
import nucleus
from nucleus import NucleusClient, DatasetItem, Frame, LidarScene, CuboidAnnotation, Point3D

In [3]:
PUBLIC_PANDASET_BUCKET = "pandaset-public"

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
paginator = s3.get_paginator('list_objects_v2')
result = paginator.paginate(Bucket=PUBLIC_PANDASET_BUCKET)

In [59]:
# Get paths for all S3 objects
object_paths = []
for page in result:
    if "Contents" in page:
        for key in page["Contents"]:
            object_path = key["Key"]
            object_paths.append(object_path)

## Data Exploration

Here's how our dataset is structured in S3. At the outermost level, we have scenes (`001`, `002`, etc). Within each scene, we have the directories `camera`, `lidar`, and `annotations`. Within camera, we have directories for different sensors (i.e. `front_camera`, `back_camera`, etc). At the innermost level, we have files containing images (jpg), pointclouds (json), or cuboid annotations (json) corresponding to a given frame.
    
```
001/
  lidar/
    00.json
    01.json
        ...
    79.json
  camera/
    back_camera/
        00.jpg
        01.jpg
        ...
        79.jpg
    front_camera/
        00.jpg
        01.jpg
        ...
        79.jpg
    ...
  annotations/
    cuboids/
        00.json
        01.json
            ...
        79.json
002/
  ...
```

In [49]:
# Define some helper functions

def is_image_path(object_path: str):
    return "camera/" in object_path and ".jpg" in object_path

def is_pointcloud_path(object_path: str):
    return "lidar/" in object_path and ".json" in object_path
  
def is_cuboid_path(object_path: str):
    return "cuboids/" in object_path and ".json" in object_path

def read_json(path: str):
    s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
    content_object = s3.Object(BUCKET, path)
    file_content = content_object.get()['Body'].read().decode('utf-8')
    return json.loads(file_content)

In [48]:
image_paths = []
pointcloud_paths = []
cuboid_paths = []

# Get paths for all images, pointclouds, and annotations in S3
for path in object_paths:
    if is_image_path(path):
        image_paths.append(path)
    elif is_pointcloud_path(path):
        pointcloud_paths.append(path)
    elif is_cuboid_path(path):
        cuboid_paths.append(path)

In [7]:
print(image_paths)
print(pointcloud_paths)
print(cuboid_paths)

['pandaset_0/001/camera/back_camera/00.jpg', 'pandaset_0/001/camera/back_camera/01.jpg', 'pandaset_0/001/camera/back_camera/02.jpg', 'pandaset_0/001/camera/back_camera/03.jpg', 'pandaset_0/001/camera/back_camera/04.jpg', 'pandaset_0/001/camera/back_camera/05.jpg', 'pandaset_0/001/camera/back_camera/06.jpg', 'pandaset_0/001/camera/back_camera/07.jpg', 'pandaset_0/001/camera/back_camera/08.jpg', 'pandaset_0/001/camera/back_camera/09.jpg', 'pandaset_0/001/camera/back_camera/10.jpg', 'pandaset_0/001/camera/back_camera/11.jpg', 'pandaset_0/001/camera/back_camera/12.jpg', 'pandaset_0/001/camera/back_camera/13.jpg', 'pandaset_0/001/camera/back_camera/14.jpg', 'pandaset_0/001/camera/back_camera/15.jpg', 'pandaset_0/001/camera/back_camera/16.jpg', 'pandaset_0/001/camera/back_camera/17.jpg', 'pandaset_0/001/camera/back_camera/18.jpg', 'pandaset_0/001/camera/back_camera/19.jpg', 'pandaset_0/001/camera/back_camera/20.jpg', 'pandaset_0/001/camera/back_camera/21.jpg', 'pandaset_0/001/camera/back_cam

### Step 3: Construct LidarScenes



In [36]:
BUCKET = "pandaset-public"
S3_BUCKET = "s3://pandaset-public"

LIDAR_SENSOR = "lidar"
CAMERA_SENSORS = ["back_camera", "front_camera", "front_left_camera", "front_right_camera", "left_camera", "right_camera"]

In [45]:
# Define some more helper functions

def get_paths_for_scene(paths: List[str], scene_dir: str):
    return [path for path in paths if f"{scene_dir}/" in path]

def extract_sensor_and_frame_from_path(path: str):
    tokens = re.split('/|\.', path) # split on / and . in file path
    sensor_name = tokens[-3]
    frame_index = int(tokens[-2])
    
    return sensor_name, frame_index

def gen_item_url(item_path: str):
    return os.path.join(S3_BUCKET, item_path)

def gen_item_reference_id(scene_id: str, frame_index: int, sensor_name: str):
    return f"scene-{scene_id}-frame-{frame_index}-{sensor_name}"

def gen_scene_reference_id(scene_id: str):
    return f"scene-{scene_id}"

In [37]:
def construct_camera_params(scene_dir: str, sensor: str):
    base_path = f"pandaset_0/{scene_dir}/camera/{sensor}/"
    intrinsics_path = os.path.join(base_path, 'intrinsics.json')
    poses_path = os.path.join(base_path, 'poses.json')
    
    intrinsics = read_json(intrinsics_path)
    poses = read_json(poses_path)
    camera_params = {"intrinsics": intrinsics, "poses": poses}
    
    return camera_params

def construct_camera_params_dict(scene_dir: str):
    """
    Constructs a dictionary of camera parameters (fx, fy, cx, cy, position, heading) for each camera sensor.
    Returns a dictionary mapping camera sensor to camera parameters.
    
    Parameters
    ----------
    scene_dir: The name of the directory containing the scene

    Returns
    -------
    Dict[str, dict]: A dictionary from camera sensor name to camera parameters dictionary
    """
    camera_sensor_to_params = {}
    for sensor in CAMERA_SENSORS:
        camera_params = construct_camera_params(scene_dir, sensor)
        camera_sensor_to_params[sensor] = camera_params
    
    return camera_sensor_to_params

In [41]:
def construct_image_item(image_path: str, sensor_name: str, frame_index: int, scene_id: str, camera_sensor_to_params: dict):
    """
    Constructs an image DatasetItem from the image stored at image_path.
    
    Parameters
    ----------
    image_path: The path of the image in S3
    sensor_name: The name of the sensor associated with the image
    frame_index: The index of the frame associated with the image
    scene_id: The name of the scene that includes the image
    camera_sensors_to_params: A dictionary from camera sensor to params

    Returns
    -------
    DatasetItem: The image DatasetItem to be added to the scene
    """
    image_url = gen_item_url(image_path)
    reference_id = gen_item_reference_id(scene_id, frame_index, sensor_name)
    
    params = camera_sensor_to_params[sensor_name]
    pose = params["poses"][frame_index]
    camera_params = {**params["intrinsics"], **pose}
    metadata = {"camera_params": camera_params}
    
    return DatasetItem(image_location=image_url, reference_id=reference_id, metadata=metadata)

def construct_pointcloud_item(pointcloud_path: str, sensor_name: str, frame_index: int, scene_id: str):
    """
    Constructs a pointcloud DatasetItem from the pointcloud stored at pointcloud_path.
    
    Parameters
    ----------
    pointcloud_path: The path of the pointcloud in S3
    sensor_name: The name of the lidar sensor associated with the pointcloud
    frame_index: The index of the frame associated with the pointcloud
    scene_id: The name of the scene that includes the pointcloud

    Returns
    -------
    DatasetItem: The pointcloud DatasetItem to be added to the scene
    """
    pointcloud_url = gen_item_url(pointcloud_path)
    reference_id = gen_item_reference_id(scene_id, frame_index, sensor_name)
    
    return DatasetItem(pointcloud_location=pointcloud_url, reference_id=reference_id)

In [30]:
def construct_scene(scene_id: str):
    """
    Constructs a LidarScene from the images and pointclouds of a scene in PandaSet.
    
    Parameters
    ----------
    scene_id: The name of the scene from PandaSet

    Returns
    -------
    LidarScene: A LidarScene containing a sequence of frames corresponding to the sensor data of a PandaSet scene
    """
    scene_reference_id = gen_scene_reference_id(scene_id)
    scene = LidarScene(scene_reference_id)

    # Construct dictionary of camera params for each camera sensor
    camera_sensor_to_params = construct_camera_params_dict(scene_id)

    # Construct image DatasetItems and add to scene
    image_paths_in_scene = get_paths_for_scene(image_paths, scene_id)
    for image_path in image_paths_in_scene:
        sensor_name, frame_index = extract_sensor_and_frame_from_path(image_path)
        image_item = construct_image_item(image_path, sensor_name, frame_index, scene_id, camera_sensor_to_params)
        scene.add_item(frame_index, sensor_name, image_item)
    
    # Construct pointcloud DatasetItems and add to scene
    pointcloud_paths_in_scene = get_paths_for_scene(pointcloud_paths, scene_id)
    for pointcloud_path in pointcloud_paths_in_scene:
        sensor_name, frame_index = extract_sensor_and_frame_from_path(pointcloud_path)
        pointcloud_item = construct_pointcloud_item(pointcloud_path, sensor_name, frame_index, scene_id)
        scene.add_item(frame_index, sensor_name, pointcloud_item)
    
    return scene

def construct_pandaset_scenes(scene_ids: List[str]):
    scenes = []
    for scene_id in scene_ids:
        scene = construct_scene(scene_id)
        scenes.append(scene)

    return scenes

In [31]:
# For this demo, we will upload scenes 001, 006, and 023 from PandaSet
SCENE_IDS = ["001", "006", "023"]
scenes = construct_pandaset_scenes(SCENE_IDS)

In [32]:
scene_1 = scenes[0]
print("number of lidar DatasetItems:", len(scene_1.get_items_from_sensor("lidar")))
print("number of DatasetItems:", len(scene_1.get_items()))
print("number of frames:", scene_1.length)
print("number of sensors:", scene_1.num_sensors)
print("sensors:", scene_1.get_sensors())

number of lidar DatasetItems: 80
number of DatasetItems: 560
number of frames: 80
number of sensors: 7
sensors: ['left_camera', 'back_camera', 'front_right_camera', 'front_left_camera', 'right_camera', 'front_camera', 'lidar']


### Step 4: Append Scenes to Dataset

In [34]:
API_KEY = "live_318209d04e3746dbafbe1f195a4a1872" # YOUR API KEY HERE
TEST_DATASET_NAME = "pandaset_3d"

In [53]:
client = NucleusClient(API_KEY, use_notebook=True)
dataset = client.create_dataset(TEST_DATASET_NAME)

In [51]:
append_job = dataset.append(scenes, asynchronous=True)
print(append_job)

AsyncJob(job_id='job_c4dkzmp81a5007g8t6r0', job_last_known_status='Running', job_type='uploadLidarScene', job_creation_time='2021-08-17T04:49:22.405Z', client=NucleusClient(api_key='live_318209d04e3746dbafbe1f195a4a1872', use_notebook=False, endpoint='https://api.scale.com/v1/nucleus'))


In [52]:
append_job.sleep_until_complete()
print(append_job.status())

Status at Mon Aug 16 21:49:27 2021: {'job_id': 'job_c4dkzmp81a5007g8t6r0', 'status': 'Running', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}
Status at Mon Aug 16 21:49:32 2021: {'job_id': 'job_c4dkzmp81a5007g8t6r0', 'status': 'Running', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}
Status at Mon Aug 16 21:49:38 2021: {'job_id': 'job_c4dkzmp81a5007g8t6r0', 'status': 'Running', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}
Status at Mon Aug 16 21:49:43 2021: {'job_id': 'job_c4dkzmp81a5007g8t6r0', 'status': 'Running', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}
Status at Mon Aug 16 21:49:48 2021: {'job_id': 'job_c4dkzmp81a5007g8t6r0', 'status': 'Running', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}
Status at Mon Aug 16 21:49:53 2021: {'job_id': 'job_c4dkzmp81a5007g8t6r0', 'status': 'Running', 'message': {}, 'job_progress': 'Na

### Step 5: Upload Annotations

In [50]:
def construct_cuboid_annotation(cuboid: dict, reference_id: str):
    """
    Constructs a CuboidAnnotation from a dictionary representation of a cuboid. 
    
    Parameters
    ----------
    cuboid: Dictionary containing label, position, dimensions, yaw, annotation_id, and metadata
    reference_id: User-defined identifier of the DatasetItem to associate this annotation with

    Returns
    -------
    CuboidAnnotation: A CuboidAnnotation corresponding to the DatasetItem referenced by reference_id
    """
    position_json = cuboid["geometry"]["position"]
    position = Point3D(position_json["x"], position_json["y"], position_json["z"])
    
    dimensions_json = cuboid["geometry"]["dimensions"]
    dimensions = Point3D(dimensions_json["x"], dimensions_json["y"], dimensions_json["z"])
    
    yaw = cuboid["geometry"]["yaw"]
    
    return CuboidAnnotation(
        label=cuboid["label"],
        position=position,
        dimensions=dimensions,
        yaw=yaw,
        reference_id=reference_id,
        annotation_id=cuboid["annotation_id"],
        metadata=cuboid["metadata"]
    )

def construct_cuboid_annotations_per_frame(cuboids_per_frame_path: str):
    """
    Constructs a list of CuboidAnnotations corresponding to the pointcloud at a given frame in the scene.
    
    Parameters
    ----------
    cuboids_per_frame_path: Path to a list of cuboids for a given lidar pointcloud

    Returns
    -------
    List[CuboidAnnotation]: A list of CuboidAnnotations representing all annotations for a pointcloud
    """
    _, frame_index = extract_sensor_and_frame_from_path(cuboids_per_frame_path) # ignore sensor since path to annotations is different
    reference_id = gen_item_reference_id(SCENE_ID, frame_index, LIDAR_SENSOR)
    cuboids_per_frame = read_json(cuboids_per_frame_path)
    
    return [construct_cuboid_annotation(cuboid, reference_id) for cuboid in cuboids_per_frame]

In [38]:
def construct_cuboid_annotations_per_scene(scene_id: str):
    cuboid_annotations = []
    cuboid_paths_in_scene = get_paths_for_scene(cuboid_paths, scene_id)
    for cuboids_per_frame_path in cuboid_paths_in_scene:
        cuboid_annotations_per_frame = construct_cuboid_annotations_per_frame(cuboids_per_frame_path)
        cuboid_annotations.extend(cuboid_annotations_per_frame)
    
    return cuboid_annotations

In [39]:
# Let's upload cuboid annotations for scene 23
SCENE_ID = "023"
annotations = construct_cuboid_annotations_per_scene(SCENE_ID)

In [56]:
print(len(annotations))
ann = annotations[0]
print(ann)

12637
CuboidAnnotation(label='Car', position=Point3D(x=-22.581, y=-35.652, z=0.776), dimensions=Point3D(x=1.935, y=4.858, z=1.851), yaw=-0.9474790371, reference_id='scene-023-frame-0-lidar', item_id=None, annotation_id='a494348f-26f4-4559-a0ae-132d4d753159', metadata={'stationary': True, 'camera_used': 5, 'attributes.object_motion': 'Parked', 'cuboids.sibling_id': '-', 'cuboids.sensor_id': -1, 'attributes.pedestrian_behavior': None, 'attributes.pedestrian_age': None, 'attributes.rider_status': None})


In [55]:
annotate_job = dataset.annotate(annotations, asynchronous=True)
print(annotate_job)

100%|██████████| 3/3 [00:12<00:00,  4.09s/it]

{'dataset_id': 'ds_c4dkzky5vjbg0d1m6fpg', 'annotations_processed': 1685, 'annotations_ignored': 0}


In [22]:
annotate_job.sleep_until_complete()
print(annotate_job.status())

Status at Mon Aug 16 17:57:26 2021: {'job_id': 'job_c4detm2tt8400bh67d30', 'status': 'Running', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}
Status at Mon Aug 16 17:57:31 2021: {'job_id': 'job_c4detm2tt8400bh67d30', 'status': 'Completed', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}
{'job_id': 'job_c4detm2tt8400bh67d30', 'status': 'Completed', 'message': {}, 'job_progress': 'NaN', 'completed_steps': 0, 'total_steps': 0}
